In [42]:
import pdfplumber
import fitz
import camelot
import warnings , math, collections
import pickle

warnings.filterwarnings("ignore", category=UserWarning) 

#path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"
path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"


pickle_path_text = r"\output\pkl\textual_data.pkl"
pickle_path_tab = r"\output\pkl\tabular_data.pkl"
samco_path = r"\files\SamcoFactSheet2024.pdf"
tata_path = r"\files\TataFactSheet2024.pdf"

dry_run_path = r"\output\DryRun.pdf"

IMPORTANT CLEANING FUNCTIONS

In [48]:
#Remove Redundant Content
removeContent =[
    'Mutual fund investments are subject to market risks, read all scheme related documents carefully.',
    '(An open-ended scheme investing across large cap, midcap and small cap stocks)',
    "(An open-ended Equity Linked Saving Scheme with a statutory lock-in of 3 years and tax benefit.)",
    "(An open-ended dynamic equity scheme investing across large cap, mid cap, small cap stocks)",
    "(An open-ended equity scheme following momentum theme)",
    "(An open-ended equity scheme following special situations theme)",
    ".",
    "st"
]

textDirection = [(1.0,0), (-1.0,0),(0,1.0),(0,-1.0)]

def extract_rgb(color_int):
    red = (color_int >> 16) & 0xFF
    green = (color_int >> 8) & 0xFF
    blue = color_int & 0xFF
    return (int(red), int(green), int(blue))
    
def adjust_color_if_white(rgb):
    # Define threshold for white detection
    white_threshold = 230
    if all(component >= white_threshold for component in rgb):
        return (255, 165, 0)  # RGB for Orange
    return rgb

def is_white_or_shade(color):
    """ Determine if the color is white or a shade close to white """
    threshold = 240  # Define how close to white the color must be
    return all(c >= threshold for c in color)


def adjust_bbox(bbox, direction, pixels):
    """
    Adjusts the boundary of a bounding box in the specified direction by a given number of pixels.

    Args:
        bbox (tuple): The original bounding box (x0, y0, x1, y1).
        direction (str): Direction to adjust ('top', 'bottom', 'left', 'right').
        pixels (int): The number of pixels to adjust by. Use positive values to expand and negative to contract.

    Returns:
        fitz.Rect: A new fitz.Rect object with the adjusted bounding box.
    """
    x0, y0, x1, y1 = bbox

    if direction == 'top':
        y0 -= pixels
    elif direction == 'bottom':
        y1 += pixels
    elif direction == 'left':
        x0 -= pixels
    elif direction == 'right':
        x1 += pixels
    else:
        raise ValueError("Invalid direction. Choose from 'top', 'bottom', 'left', 'right'.")

    adjusted_rect = fitz.Rect(x0, y0, x1, y1)
    return adjusted_rect

document = fitz.open(path +tata_path)
total_pages = document.page_count
document.close()
for i in range(1,total_pages+1):
    removeContent.append(str(i))


PDF DATA SEGREGATION AND EXTRACTION

In [59]:
def extract_pdf_blocks(input_pdf_path):
    """Open the PDF and extract all blocks of text, images, and other content.
    Args:input_pdf_path (str): Path to the input PDF.
    Returns:list: A list of pages, where each page is a dictionary containing blocks of content.
    """
    
    blocks_data = []
    
    input_doc = fitz.open(input_pdf_path)
    

    with pdfplumber.open(input_pdf_path) as pdf:
        for page_number, pdf_page in enumerate(pdf.pages):
            # Extract blocks of content using PyMuPDF
            doc_page = input_doc[page_number]
            blocks = doc_page.get_text("dict")["blocks"]
            
            #get the color and size for each block then aggregate
            page_color = []
            page_size = []
            
            for block in blocks:
                if 'lines' not in block:
                    continue
                for lines in block['lines']:
                    for span in lines['spans']:
                        
                        #aggregate operations
                        color = extract_rgb(span['color'])
                        color = adjust_color_if_white(color)
                        size = span['size']
                        page_color.append(color)
                        page_size.append(size)
                        
            page_color = collections.Counter(page_color)
            page_size = collections.Counter(page_size)
            
            
            table_bboxes = []
            for table_bbox in [table.bbox for table in pdf_page.find_tables()]:
                table_bbox = adjust_bbox(table_bbox, 'top', 10)
                table_bboxes.append(table_bbox)
                
            # Append the blocks along with table bounding boxes from pdfplumber
            blocks_data.append({
                "blocks": blocks,
                "table_bboxes": table_bboxes,
                "page_rect": doc_page.rect,
                "page_colors":dict(page_color),
                "page_sizes": dict(page_size),
                "total_pages": input_doc.page_count
            })

    input_doc.close()
    return blocks_data

In [61]:
def remove_image_blocks_and_create_pdf(blocks_data, output_pdf_path):
    """
    Returns:
        list: Updated block data without image blocks.
    """
    output_doc = fitz.open()
    output_blocks_data = []
    image_blocks_data = []

    for page_data in blocks_data:
        page_rect = page_data["page_rect"]
        blocks = page_data["blocks"]
        tablebbox = page_data['table_bboxes']
        page_colors = page_data['page_colors']
        page_sizes = page_data['page_sizes']
        

        # Create a new page in the output PDF
        output_page = output_doc.new_page(width=page_rect.width, height=page_rect.height)

        page_image_data = []
        

        for block in blocks:
            if "image" in block.keys():
                page_image_data.append(block)
                del block['image']
                continue #skip the image box
                # bbox = block.get('bbox',[0,0,0,0])
                # try:
                #     output_page.draw_rect(
                #         bbox,  # Bounding box of the rectangle
                #         color=(0.9, 0.9, 0.9),  # Light grey color
                #         fill=True,  # Fill the rectangle
                #     )
                # except Exception:
                #     continue  # Skip if there's an error drawing the rectangle
                # continue  # Skip further processing for image blocks


            if "lines" in block:  # Process text lines
                for line in block["lines"]:
                    for span in line["spans"]:
                        bbox = span.get("bbox", [0, 0, 0, 0])
                        text = span["text"]
                        size = float(span.get("size", 12)) # Default font size if not provided
                        color = span.get("color", (0, 0, 0))  # Default color (black)
                        font = span.get("font", "helv").lower()  # Fallback to 'helv' if font is not found

                        # Check if the font is bold or italic and modify font name accordingly
                        fontname = "helv"  # Default font
                        if "bold" in font or "light" in font:
                            fontname += '-b'# Add suffix for bold
                        if "italic" in font:
                            fontname += '-i'# Add suffix for italic

                        # Normalize color if it's in integer form (e.g., 0x000000)
                        if isinstance(color, int):
                            color = (
                                (color >> 16) & 255, (color >> 8) & 255, color & 255
                            )
                            
                        if is_white_or_shade(color):
                            color = (255, 165, 0)  # Change color to orange
                        
                        color = tuple(c / 255 for c in color)  # Convert to 0-1 range

                        try:
                            output_page.insert_text(
                                (bbox[0], bbox[1]),
                                text,
                                fontsize=size,
                                fontname=fontname,
                                color=color,
                            )
                        except Exception:
                            output_page.insert_text(
                                (bbox[0], bbox[1]),
                                text,
                                fontsize=size,
                                fontname="helv",  # Fallback font
                                color=color,
                            )
        # Append instance each instance of image to 
        image_blocks_data.append(page_image_data)
        
        # After processing the page, extract text blocks from the newly created output page
        output_blocks = output_page.get_text("dict")["blocks"]
        output_blocks_data.append({
            "blocks": output_blocks,
            "page_rect": page_rect,
            "table_bboxes": tablebbox,
            "page_colors": page_colors,
            "page_sizes": page_sizes
            })


    # Save the output PDF
    output_doc.save(output_pdf_path)
    output_doc.close()

    return output_blocks_data, image_blocks_data

In [60]:
def create_tabular_pdf(blocks_data, output_pdf_path):
    """Returns:list: Block data containing only tabular content."""
    
    output_doc = fitz.open()
    tabular_blocks = []
    
    for page_data in blocks_data:
        page_rect = page_data["page_rect"]
        blocks = page_data["blocks"]
        table_bboxes = page_data["table_bboxes"]

        # Create a new page in the output PDF
        output_page = output_doc.new_page(width=page_rect.width, height=page_rect.height)

        page_tabular_blocks = []
        written_texts = set()  # To track written spans (text + bbox)

        for block in blocks:
            is_tabular_block = False  # Track if the block belongs to a table
            block_key = hash(str(block.get("bbox", "")) + str(block.get("lines", [])))  # Unique key for the block

            # Avoid reprocessing the same block
            if block_key in written_texts:
                continue

            if "lines" in block.keys():  # Process text blocks
                for line in block["lines"]:
                    for span in line["spans"]:
                        bbox = span.get("bbox", [0, 0, 0, 0])
                        text = span["text"].strip()  # Normalize text

                        # Check if the text is within any table bbox
                        inside_table = any(
                            bbox[0] >= table_bbox[0] and
                            bbox[1] >= table_bbox[1] and
                            bbox[2] <= table_bbox[2] and
                            bbox[3] <= table_bbox[3]
                            for table_bbox in table_bboxes
                        )

                        if inside_table:
                            # Create a unique identifier for this span
                            span_key = (text, tuple(map(round, bbox)))

                            if span_key not in written_texts:
                                written_texts.add(span_key)  # Mark this span as written
                                is_tabular_block = True  # Mark the block as part of a table

                                # Insert text into the PDF
                                size = float(span.get("size", 12))
                                color = span.get("color", (0, 0, 0))
                                font = span.get("font", "helv").lower()

                                # Normalize color
                                if isinstance(color, int):
                                    color = (
                                        (color >> 16) & 255, (color >> 8) & 255, color & 255
                                    ) 
                                    
                                if is_white_or_shade(color):
                                    color = (255, 165, 0)  # Change color to orange
                                color = tuple(c / 255 for c in color)

                                # Write text to the PDF
                                try:
                                    output_page.insert_text(
                                        (bbox[0], bbox[1]),
                                        text,
                                        fontsize=size,
                                        fontname=font,
                                        color=color,
                                    )
                                except Exception:
                                    output_page.insert_text(
                                        (bbox[0], bbox[1]),
                                        text,
                                        fontsize=size,
                                        fontname="helv",
                                        color=color,
                                    )

            if is_tabular_block:
                page_tabular_blocks.append(block)
                written_texts.add(block_key)  # Mark the block as processed

        # Draw table bounding boxes with light green color
        light_green = (0.2, 1, 0.5)  # RGB for light green
        for table_bbox in table_bboxes:
            #table_bbox = adjust_bbox(table_bbox,"top", 20)
            rect = fitz.Rect(table_bbox)
            output_page.draw_rect(rect, color=light_green, width=.5)
        
        # Append only unique tabular blocks for this page
        tabular_blocks.append(page_tabular_blocks)
        
    tabular_blocks_data = {
        "blocks": tabular_blocks,
        "page_rect": page_rect,
        "table_bboxes": table_bboxes,
    }

    # Save the output PDF
    output_doc.save(output_pdf_path)
    output_doc.close()

    return tabular_blocks_data

In [58]:
def create_non_tabular_pdf(blocks_data, output_pdf_path):
    
    """Returns:list: Block data containing only non-tabular content."""
    
    output_doc = fitz.open()
    non_tabular_blocks = []
    

    for page_data in blocks_data:
        page_rect = page_data["page_rect"]
        blocks = page_data["blocks"]
        table_bboxes = page_data["table_bboxes"]

        # Create a new page in the output PDF
        output_page = output_doc.new_page(width=page_rect.width, height=page_rect.height)

        page_non_tabular_blocks = []
        written_texts = set()  # To track written spans (text + bbox)
        written_blocks = set()  # To track processed blocks

        for block in blocks:
            block_key = hash(str(block.get("bbox", "")) + str(block.get("lines", [])))  # Unique key for block

            # Avoid reprocessing the same block
            if block_key in written_blocks:
                continue

            is_non_tabular_block = False  # Track if the block is non-tabular

            if "lines" in block:  # Process text blocks
                for line in block["lines"]:
                    for span in line["spans"]:
                        bbox = span.get("bbox", [0, 0, 0, 0])
                        text = span["text"].strip()  # Normalize text
                        
                        #set important conditions
                        condition = (text not in removeContent and 
                                     line['dir'] in textDirection)
                        
                        if condition:
                            # Check if the text is within any table bbox
                            inside_table = any(
                                bbox[0] >= table_bbox[0] and
                                bbox[1] >= table_bbox[1] and
                                bbox[2] <= table_bbox[2] and
                                bbox[3] <= table_bbox[3]
                                for table_bbox in table_bboxes
                            )

                            if not inside_table:
                                # Create a unique identifier for this span
                                span_key = (text, tuple(map(round, bbox)))

                                if span_key not in written_texts:
                                    written_texts.add(span_key)  # Mark this span as written
                                    is_non_tabular_block = True  # Mark the block as non-tabular

                                    # Insert text into the PDF
                                    size = float(span.get("size", 12))
                                    color = span.get("color", (0, 0, 0))
                                    font = span.get("font", "helv").lower()

                                    # Normalize color
                                    if isinstance(color, int):
                                        color = (
                                            (color >> 16) & 255, (color >> 8) & 255, color & 255
                                        )
                                    
                                    if is_white_or_shade(color):
                                        color = (255, 165, 0)  # Change color to orange
                                    
                                    color = tuple(c / 255 for c in color)

                                    # Write text to the PDF
                                    try:
                                        output_page.insert_text(
                                            (bbox[0], bbox[1]),
                                            text,
                                            fontsize=size,
                                            fontname=font,
                                            color=color,
                                        )
                                    except Exception:
                                        output_page.insert_text(
                                            (bbox[0], bbox[1]),
                                            text,
                                            fontsize=size,
                                            fontname="helv",
                                            color=color,
                                        )

            if is_non_tabular_block:
                page_non_tabular_blocks.append(block)
                written_blocks.add(block_key)  # Mark this block as processed
                
                
        light_green = (0.2, 1, 0.3)  # RGB for light green
        for table_bbox in table_bboxes:
            #table_bbox = adjust_bbox(table_bbox,"top", 20)
            rect = fitz.Rect(table_bbox)
            output_page.draw_rect(rect, color=light_green, width=.5)

        # Append only unique non-tabular blocks for this page
        non_tabular_blocks.append(page_non_tabular_blocks)
    
    non_tabular_blocks_data = {
        "blocks": non_tabular_blocks,
        "page_rect": page_rect,
        "table_bboxes": table_bboxes,
    }

    # Save the output PDF
    output_doc.save(output_pdf_path)
    output_doc.close()

    return non_tabular_blocks_data

DRY RUN CODE

In [76]:
#samco output path
no_image_path = path +r"\output\NoImgPdf.pdf"
textual_pdf_path = path + r"\output\TextualPdf.pdf"
tabular_pdf_path = path + r"\output\TabularPdf.pdf"

In [37]:
#tata output path
no_image_path = path +r"\output\TatanoImgPdf.pdf"
textual_pdf_path = path + r"\output\TatatextalPdf.pdf"
tabular_pdf_path = path + r"\output\TatatabularPdf.pdf"

In [63]:
blocks_data = extract_pdf_blocks(path + samco_path)
text_data, image_data = remove_image_blocks_and_create_pdf(blocks_data, no_image_path)
print("\n Success !!")


 Success !!


In [77]:
#blk_data = blocks_data #pdf_blocks
# blk_data = text_data #pdf minus images
# print(len(blk_data))
# for pgn, page in enumerate(blk_data):
#     print(f"\nPage:{pgn}")
#     for block in page['blocks']:
#        print(f"\n{block}")

In [64]:
textual_block_data = create_non_tabular_pdf(blocks_data, textual_pdf_path)
# print("\nCode Successful")

tabular_block_data = create_tabular_pdf(text_data, tabular_pdf_path)
print("\nCode Successful")


Code Successful


In [29]:
with open(path + pickle_path_text , 'wb') as file:
    pickle.dump(textual_block_data, file)
    
with open(path + pickle_path_tab , 'wb') as file:
    pickle.dump(tabular_block_data, file)

In [ ]:
"""Open the pdf , get all text data and blocks and draw a boundary along each boundary boxes
    Args:input_pdf_path(str) , output_pdf_path (str)
    Returns: nothing, a new pdf created
"""
def draw_boundaries_on_pdf(input_pdf_path, output_pdf_path):
    # Open the PDF file
    doc = fitz.open(input_pdf_path)
    for page in doc:
        blocks = page.get_text("blocks")  # Get the blocks of text on the page
        for block in blocks:
            bbox = block[:4]  # The bbox is the first four elements of the block
            # Draw a rectangle with an orange border around the bbox
            page.draw_rect(bbox, color=(1.0, 0.647, 0.0), width=1.5, overlay=False)
    
    # Save the modified document to a new file
    doc.save(output_pdf_path)
    doc.close()
# Use the function with your file paths
draw_boundaries_on_pdf(path + samco_path, path + dry_run_path)

In [77]:
def draw_table_boundaries(input_pdf_path, output_pdf_path):
    with pdfplumber.open(input_pdf_path) as pdf:
        doc = fitz.open(input_pdf_path)
        for page_number, page in enumerate(pdf.pages):
            fitz_page = doc[page_number]
            tables = page.find_tables()
            for table in tables:
                bbox = table.bbox
                rect = fitz.Rect(bbox[0], bbox[1], bbox[2], bbox[3])
                fitz_page.draw_rect(rect, color=(0, 0, 1), width=1.5, overlay=False)
        doc.save(output_pdf_path)
        doc.close()

draw_table_boundaries(no_image_path, path + dry_run_path)

In [17]:
def extract_tables(blocks):
    """
    Extracts tables from a list of fitz blocks.

    Args:
        blocks: List of blocks extracted from the PDF using PyMuPDF.

    Returns:
        A list of Camelot Table objects.
    """

    # 1. Create a temporary string to hold the table data
    table_text = ""
    for block in blocks:
        if "lines" in block:  # Check if the block is text
            for line in block['lines']:
                for span in line['spans']:
                    table_text += span['text']  # Extract the text from the span
                table_text += "\n"  # Add a newline after each line

    path = r"C:\Users\rando\OneDrive\Documents\mywork-repo\output\temp_table.txt"
    with open(path, 'w') as f:
        f.write(table_text)

    try:
        tables = camelot.read_pdf('temp_table.txt', flavor='lattice', pages='1')
        return tables
    except Exception as e:
        print(f"Error extracting tables: {e}")
        return []